In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, lit,sum

In [2]:
df=pd.read_json('/content/orders_dataset.json')
df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [3]:
df_order = pd.read_json('/content/order_items_dataset.json')
df_order.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [4]:
df_product=pd.read_json('/content/products_dataset.json')
df_product.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [5]:
df1=pd.merge(df_order,df, on='order_id', how="inner")
df1.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29 00:00:00
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15 00:00:00
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05 00:00:00
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20 00:00:00
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17 00:00:00


In [6]:
merged_df= pd.merge(df1,df_product
 , on='product_id', how="inner")
merged_df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,order_delivered_customer_date,order_estimated_delivery_date,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,2017-09-20 23:43:48,2017-09-29 00:00:00,cool_stuff,58,598,4,650,28,9,14
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,2017-05-12 16:04:24,2017-05-15 00:00:00,pet_shop,56,239,2,30000,50,30,40
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,2018-01-22 13:19:16,2018-02-05 00:00:00,moveis_decoracao,59,695,2,3050,33,13,33
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,2018-08-14 13:32:39,2018-08-20 00:00:00,perfumaria,42,480,1,200,16,10,15
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,2017-03-01 16:42:31,2017-03-17 00:00:00,ferramentas_jardim,59,409,1,3750,35,40,30


In [7]:
merged_df['order_status'].unique()

array(['delivered', 'shipped', 'canceled', 'invoiced', 'processing',
       'approved', 'unavailable'], dtype=object)

In [8]:
merged_df.to_csv('merged_df.csv', index=False)

In [9]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 22 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       112650 non-null  object        
 1   order_item_id                  112650 non-null  int64         
 2   product_id                     112650 non-null  object        
 3   seller_id                      112650 non-null  object        
 4   shipping_limit_date            112650 non-null  object        
 5   price                          112650 non-null  float64       
 6   freight_value                  112650 non-null  float64       
 7   customer_id                    112650 non-null  object        
 8   order_status                   112650 non-null  object        
 9   order_purchase_timestamp       112650 non-null  object        
 10  order_approved_at              112635 non-null  datetime64[ns]
 11  

In [10]:
spark = SparkSession.builder.appName("MissingValues")\
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") .getOrCreate()
spark_df = spark.createDataFrame(merged_df)

In [11]:
missing_spark_df = spark_df.select([sum(col(c).isNull().cast("int")).alias(c) for c in spark_df.columns])
missing_spark_df.show()

+--------+-------------+----------+---------+-------------------+-----+-------------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|order_id|order_item_id|product_id|seller_id|shipping_limit_date|price|freight_value|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+--------+-------------+----------+---------+-------------------+-----+-------------+-----------+------------+------------------------+-----------------+----------------------------+----------------

In [12]:
spark_df = spark_df.filter(col("order_approved_at").isNotNull())

In [13]:
spark_df = spark_df.dropDuplicates(['product_id'])

In [14]:
from pyspark.ml.feature import StringIndexer

In [15]:
Indexer_product_category_name=StringIndexer(inputCol='product_category_name',outputCol='product_category_name_index')
spark_df=Indexer_product_category_name.fit(spark_df).transform(spark_df)

In [16]:
spark_df=spark_df.drop('product_category_name')

In [17]:
spark_df.show(10,truncate=False)

+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+------+-------------+--------------------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+---------------------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |shipping_limit_date|price |freight_value|customer_id                     |order_status|order_purchase_timestamp|order_approved_at  |order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|product_category_name_index|


In [18]:
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer
from pyspark.ml.classification import LogisticRegression
data= spark_df.filter(F.col("order_status").isin(["delivered", "canceled"]))

data = data.withColumn(
    "label",
    F.when(F.col("order_status") == "delivered", 1.0).otherwise(0.0)
)

data = data.select(
    "label","order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date",
    "product_category_name_index",
    "product_weight_g",
    "price",
    "freight_value",
    "product_photos_qty",
    "seller_id",
    "customer_id"
)


In [19]:
from pyspark.sql import functions as F
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, Imputer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

data = (
    data
    .withColumn("order_purchase_timestamp", F.to_timestamp("order_purchase_timestamp", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("order_approved_at", F.to_timestamp("order_approved_at", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("order_delivered_customer_date", F.to_timestamp("order_delivered_customer_date", "yyyy-MM-dd HH:mm:ss"))
    .withColumn("order_estimated_delivery_date", F.to_timestamp("order_estimated_delivery_date", "yyyy-MM-dd HH:mm:ss"))
)

data_ml = (
    data
    .withColumn(
        "time_to_approve",
        (F.col("order_approved_at").cast("long") - F.col("order_purchase_timestamp").cast("long")).cast("double")
    )
    .withColumn(
        "delivery_time",
        (F.col("order_delivered_customer_date").cast("long") - F.col("order_approved_at").cast("long")).cast("double")
    )
    .withColumn(
        "delay_time",
        (F.col("order_delivered_customer_date").cast("long") - F.col("order_estimated_delivery_date").cast("long")).cast("double")
    )
)

numeric_cols = [
    "product_weight_g",
    "price",
    "freight_value",
    "product_photos_qty",
    "time_to_approve",
    "delivery_time",
    "delay_time"
]
for c in numeric_cols + ["product_category_name_index"]:
    if c in data_ml.columns:
        data_ml = data_ml.withColumn(c, F.col(c).cast("double"))



In [20]:
stages = []

stages.append(StringIndexer(inputCol="seller_id", outputCol="seller_id_index", handleInvalid="keep"))
stages.append(StringIndexer(inputCol="customer_id", outputCol="customer_id_index", handleInvalid="keep"))

ohe_inputs  = ["product_category_name_index", "seller_id_index", "customer_id_index"]
ohe_outputs = ["product_category_ohe", "seller_id_ohe", "customer_id_ohe"]
stages.append(OneHotEncoder(inputCols=ohe_inputs, outputCols=ohe_outputs, handleInvalid="keep"))

In [21]:
valid_numeric_cols = []
for c in numeric_cols:
    if c in data_ml.columns:
        non_null_count = data_ml.filter(F.col(c).isNotNull()).limit(1).count()
        if non_null_count > 0:
            valid_numeric_cols.append(c)

if valid_numeric_cols:
    imputer = Imputer(
        strategy="median",
        inputCols=valid_numeric_cols,
        outputCols=[f"{c}_imp" for c in valid_numeric_cols]
    )
    stages.append(imputer)
    feature_cols = [f"{c}_imp" for c in valid_numeric_cols]
else:
    feature_cols = []

for c in set(numeric_cols) - set(valid_numeric_cols):
    data_ml = data_ml.withColumn(f"{c}_imp", F.lit(0.0))
    feature_cols.append(f"{c}_imp")

feature_cols = feature_cols + ohe_outputs
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="keep")
stages.append(assembler)

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=50)
stages.append(lr)

pipeline = Pipeline(stages=stages)


train, test = data_ml.randomSplit([0.8, 0.2], seed=42)

model = pipeline.fit(train)
pred  = model.transform(test).cache()
eval_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
eval_f1  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

eval_wp  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
eval_wr  = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")

accuracy  = eval_acc.evaluate(pred)
f1        = eval_f1.evaluate(pred)
precision = eval_wp.evaluate(pred)
recall    = eval_wr.evaluate(pred)

print(f"Accuracy : {accuracy:.4f}")
print(f"F1 Score : {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall   : {recall:.4f}")


Accuracy : 0.9986
F1 Score : 0.9986
Precision: 0.9986
Recall   : 0.9986


In [26]:
df_with_pred = pred.select(
    *test.columns,
    F.col("prediction").alias("predicted_label"),

)

df_with_pred.toPandas().to_csv("predictions_vs_labels.csv", index=False)
